# Run Moatless Tools

First, index the codebase in a vector store.

Set `repo_dir` to the path of the repository you want to index.

In [1]:
from moatless.index import CodeIndex, IndexSettings
from moatless.repository import FileRepository
from moatless.workspace import Workspace

# An OPENAI_API_KEY is required to use the OpenAI Models
model = "gpt-4o-2024-05-13"
index_settings = IndexSettings(
    embed_model="text-embedding-3-small"
)

repo_dir = "/Users/aidand/dev/hello-swe-bench/django"
file_repo = FileRepository(repo_path=repo_dir)

code_index = CodeIndex(file_repo=file_repo, settings=index_settings)
nodes, tokens = code_index.run_ingestion()

print(f"Indexed {nodes} nodes and {tokens} tokens")

workspace = Workspace(file_repo=file_repo, code_index=code_index)

/Users/aidand/dev/moatless-tools/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 5806/5806 [02:26<00:00, 39.55it/s]
Generating embeddings: 0it [00:00, ?it/s]

Indexed 0 nodes and 0 tokens


In [7]:
code_index.search(query=".")

INFO:moatless.index.code_index:vector_search() Searching for query [....] and file pattern [None].
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:moatless.index.code_index:vector_search() Returning 0 search results. (Ignored 0 removed search results. Filtered out 0 search results.)
INFO:moatless.index.code_index:semantic_search() Found 0 code spans in 0 files.


SearchCodeResponse(message='Found 0 code spans.', hits=[])

Then use the `SearchLoop` to find the relevant code.

In [4]:
from moatless.transitions import search_transitions
from moatless import AgenticLoop
import logging
logging.basicConfig(level=logging.INFO)

instructions = """
Squashing migrations with Meta.index_together -> indexes transition should remove deprecation warnings.
Description

Squashing migrations with Meta.index_together -> Meta.indexes transition should remove deprecation warnings. As far as I'm aware, it's a 4.2 release blocker because you cannot get rid of the index_together deprecation warnings without rewriting migrations, see comment.
"""

transitions = search_transitions(model=model)
search_loop = AgenticLoop(transitions, workspace=workspace)

search_response = search_loop.run(instructions)
print(search_response.message)

print(workspace.file_context.create_prompt())

INFO:moatless.transitions:{<class 'moatless.find.decide.DecideRelevance'>: {'max_iterations': 5}}
INFO:Loop:Transitioning from Pending to SearchCode
INFO:moatless.index.code_index:vector_search() Searching for query [
Squashing migrations with Meta.index_together -> ...] and file pattern [None].
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:moatless.index.code_index:vector_search() Returning 0 search results. (Ignored 0 removed search results. Filtered out 0 search results.)
INFO:moatless.index.code_index:semantic_search() Found 0 code spans in 0 files.
INFO:moatless.trajectory:Getting transitions for SearchCode from 0 transitions.
INFO:Loop:SearchCode Create completion with 2 messages
14:53:40 - LiteLLM:INFO: utils.py:2794 - 
LiteLLM completion() model= gpt-4o-2024-05-13; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-2024-05-13; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "H

The loop was aborted because the number of retries exceeded the limit.



Execute the `CodeLoop` to apply the changes.

In [6]:
from moatless.transitions import code_transitions

code_loop = AgenticLoop(transitions=code_transitions(), workspace=workspace)
code_response = code_loop.run(instructions)

print(f"Response: {code_response.message}")

Response: The token limit check has been successfully removed from the `completion` function in moatless/llm/completion.py.


Run a `$ git diff` to see the changes.

In [7]:
import subprocess

output = subprocess.run(
      ["git", "diff"],
      capture_output=True,
      text=True,
      cwd=repo_dir,
)

print(output.stdout)

diff --git a/moatless/llm/completion.py b/moatless/llm/completion.py
index 2c95e47..a926948 100644
--- a/moatless/llm/completion.py
+++ b/moatless/llm/completion.py
@@ -48,10 +48,6 @@ def completion(
 
     metadata["trace_name"] = trace_name
 
-    tokens = token_counter(messages=messages[-1:])
-    if tokens > Settings.max_message_tokens:
-        raise ValueError(f"Too many tokens in the new message: {tokens}")
-
     response = litellm.completion(
         model=model,
         max_tokens=max_tokens,

